In [ ]:
### It seems like the HF repo is not working, i eded up downloading it manually.
from pprint import pprint
from datasets import load_dataset

# dataset_dict = load_dataset('HUPD/hupd',
#     name='sample',
#     data_files="https://huggingface.co/datasets/HUPD/hupd/blob/main/hupd_metadata_2022-02-22.feather",
#     icpr_label=None,
# )
